# Individual Socioeconomic Status and the Placement of Public Art Installations within Vancouver

In [ ]:
# load library

suppressPackageStartupMessages(
    {library(tidyr)
    library(dplyr)
    library(tidyverse)
    library(broom)
    library(car)
    library(sandwich)
    library(stargazer)
})

## outline
- Introduction**done**
- Data Description **done**
    - Summary Statistics **done**
- Model
    - Table of Results **done**
- Discussion**done**
- Conclusion**done**
- References

## Introduction

**EDIT HERE!!!!**

Introduction: the introduction is the most important part of your paper. A good
introduction should do the following:
- Provide motivation for the paper: why should I care about this paper?
- Clearly and explicitly state the research question for the paper. What is this paper
going to find out?
- Provide key background: what do we know about the answer already?
- Give an overview of your results and what your paper will find: what did you learn?

Public art plays a significant role in shaping the cultural identity and aesthetic landscape of
cities. The installations are widely used to foster public interaction, promote a sense of community, contribute to local identity, provoke thought or discussion, and tackle social exclusion (Schuermans, 2012). However, not all communities have equal access to public art, and the factors influencing its placement are not fully understood. It matters because public art affects how people experience and connect with their neighborhoods. This research aims to investigate how the levels of individual socioeconomic status in neighborhoods determine the placement of public art installations within Vancouver. We will apply regression analysis to answer this question, following the regular track for this project. After testing different models, we found that there're positive correlation between income, education attainment and art installation. 



## Data Description

To conduct the analysis, we will use two primary datasets. 

The first one is the 2021 Canadian Census, including many variables at the Census Dissemination Area (DA) level, as the source of data on the socioeconomic status of different neighborhoods in Vancouver. Another dataset is a public art dataset that includes the titles, types, status, and locations of art installations in Vancouver. We will study our research topic by combining the information from these two datasets. 

For the measures of socioeconomic status, we will focus on the income and education level of people in different neighborhoods in Vancouver. For income variables, we will use the income Gini index, also the median income. To measure educational attainment, we will use the education level. In addition to education and income, we will include the population density variable in our model as a control to test whether the other variables affect the placement of public art in Vancouver.

In [ ]:
public_art <- read_delim("data/public-art.csv")

head(public_art, n=3)

Firstly, we count the total number of art installation in each neighborhood.

In [ ]:
art_count <- public_art |>
    group_by(Neighbourhood) |>
    summarize(art_count=n())

head(art_count)

In [ ]:
load('data/data_hoods_nosf.rda')
head(census_data_hood, n=5)

We take the the number of art installations in a `Neighborhood` in Vancouver as our target variable Y. Now we need to merge this dataset with our explanatory variables Xs.

We need to convert the dummy variables in the dataset to one categorical column.

In [ ]:
dummy_cols <- c('in_neighbourhood_Arbutus.Ridge','in_neighbourhood_Grandview.Woodland','in_neighbourhood_Kensington.Cedar.Cottage',
                'in_neighbourhood_Killarney','in_neighbourhood_Kitsilano','in_neighbourhood_Strathcona','in_neighbourhood_Sunset','in_neighbourhood_West.Point.Grey',
                'in_neighbourhood_Dunbar.Southlands','in_neighbourhood_Fairview','in_neighbourhood_Mount.Pleasant','in_neighbourhood_Renfrew.Collingwood','in_neighbourhood_West.End',
                'in_neighbourhood_Downtown','in_neighbourhood_Hastings.Sunrise','in_neighbourhood_Kerrisdale','in_neighbourhood_Marpole','in_neighbourhood_Oakridge',
                'in_neighbourhood_Riley.Park','in_neighbourhood_South.Cambie','in_neighbourhood_Shaughnessy','in_neighbourhood_Victoria-Fraserview')

census_data_hood_long <- census_data_hood %>%
  pivot_longer(cols = all_of(dummy_cols), names_to = "variable", values_to = "value") %>%
  filter(value == 1) %>%
  group_by(GeoUID) %>%
  summarise(Neighbourhood = first(variable), .groups = "drop")

geouid_neighb <- census_data_hood %>%
  left_join(census_data_hood_long, by = "GeoUID") %>%
  select(GeoUID, Neighbourhood) %>%
  filter(!is.na(Neighbourhood))

In [ ]:
head(geouid_neighb,n=2)
slice(geouid_neighb,300:301)

In [ ]:
print(list(art_count$Neighbourhood))

Here the names of the neighborhood are different from those in our public art dataset `art_prop`, let us rewrite them.

In [ ]:
geouid_neighb <- geouid_neighb %>%
  mutate(Neighbourhood = Neighbourhood %>%
           gsub('in_neighbourhood_Arbutus.Ridge', 'Arbutus Ridge', .) %>%
           gsub('in_neighbourhood_Downtown', 'Downtown', .) %>%
           gsub('in_neighbourhood_Dunbar.Southlands', 'Dunbar-Southlands', .) %>%
           gsub('in_neighbourhood_Fairview', 'Fairview', .) %>%
           gsub('in_neighbourhood_Grandview.Woodland', 'Grandview-Woodland', .) %>%
           gsub('in_neighbourhood_Hastings.Sunrise', 'Hastings-Sunrise', .) %>%
           gsub('in_neighbourhood_Kensington.Cedar.Cottage', 'Kensington-Cedar Cottage', .) %>%
           gsub('in_neighbourhood_Kerrisdale', 'Kerrisdale', .) %>%
           gsub('in_neighbourhood_Killarney', 'Killarney', .) %>%
           gsub('in_neighbourhood_Kitsilano', 'Kitsilano', .) %>%
           gsub('in_neighbourhood_Marpole', 'Marpole', .) %>%
           gsub('in_neighbourhood_Mount.Pleasant', 'Mount Pleasant', .) %>%
           gsub('in_neighbourhood_Oakridge', 'Oakridge', .) %>%
           gsub('in_neighbourhood_Renfrew.Collingwood', 'Renfrew-Collingwood', .) %>%
           gsub('in_neighbourhood_Riley.Park', 'RileyPark', .) %>%
           gsub('in_neighbourhood_Shaughnessy', 'Shaughnessy', .) %>%
           gsub('in_neighbourhood_South.Cambie', 'South Cambie', .) %>%
           # gsub('in_neighbourhood_Grandview.Woodland', 'Stanley Park', .) %>%
           gsub('in_neighbourhood_Strathcona', 'Strathcona', .) %>%
           gsub('in_neighbourhood_Sunset', 'Sunset', .) %>%
           gsub('in_neighbourhood_Victoria-Fraserview', 'Victoria-Fraserview', .) %>%
           gsub('in_neighbourhood_West.End', 'West End', .) %>%
           gsub('in_neighbourhood_West.Point.Grey', 'West Point Grey', .))

In [ ]:
# check the modification
head(geouid_neighb,n=1)

In [ ]:
# load our census data
load('data/data_art_nosf.rda')

head(census_data_art, n=3)

Since we are interested in the influence of socioeconomic factors in the public art installation density in an area, the important variables for us in this dataset are the population density, median income, income gini index and certificate density in each area in Vancouver, let us select them.

For `certificate density`, we divide the original variable, the number of total highest certificates, by the population to get the density of certificates in a region. We rename the variables we are interested in making it more readable and operable.

In [ ]:
subset_data <- census_data_art %>% select("Region.Name",
                                          "Population",
                                          "v_CA21_6..Population.density.per.square.kilometre",
                                         "v_CA21_906..Median.total.income.of.household.in.2020....",
                                         "v_CA21_1140..Gini.index.on.adjusted.household.total.income",
                                         "v_CA21_5817..Total...Highest.certificate..diploma.or.degree.for.the.population.aged.15.years.and.over.in.private.households",
                                          ) |>
    rename(population_density = v_CA21_6..Population.density.per.square.kilometre,
           median_income = v_CA21_906..Median.total.income.of.household.in.2020....,
           gini_income = v_CA21_1140..Gini.index.on.adjusted.household.total.income,
           highest_certificate = v_CA21_5817..Total...Highest.certificate..diploma.or.degree.for.the.population.aged.15.years.and.over.in.private.households,
           GeoUID = Region.Name
          ) |>
    mutate(certificate_density = highest_certificate/Population,
          ) |>
    select(-highest_certificate, -Population)

head(subset_data, n=3)

Now we can merge the datasets of public art and our census, by using the `GeoUID` information in our neiborghhood dataset.

In [ ]:
subset_data_neighb <- geouid_neighb %>%
  left_join(subset_data, by = "GeoUID")

To summarize the information in each neighborhood, we take the average of our explanatory variables for each neighboorhood.

In [ ]:
clean_data <- subset_data_neighb %>%
  left_join(art_count, by = "Neighbourhood") %>%
  select(-GeoUID) %>%
  group_by(Neighbourhood) %>%
  summarize(
    mean_population_density = mean(population_density, na.rm = TRUE),
    mean_median_income = mean(median_income, na.rm = TRUE),
    mean_gini_income = mean(gini_income, na.rm = TRUE),
    mean_certificate_density = mean(certificate_density, na.rm = TRUE),
    art_count = mean(art_count, na.rm = TRUE)   # same value for the same neighborhood
  )

head(clean_data,n=3)

In [ ]:
nrow(clean_data)

This `clean_data` is our analysis sample, which contains 22 observations with 4 features of `mean_population_density`, `mean_median_income`, `mean_gini_income`, and `mean_certificate_density` and our target variable `art_count`.

The sample size is small since we are taking one neighborhood in Vancouver as the unit of our sample. There could be some problem concerning with inferential part of our research, but considering the interested population in our research question, this sample is representative enough of our population. Therefore, we may be able to approximate our estimates to the true parameter even if the result of our inferential model is not good due to the limites sample size.

Also, with this size of sample and the consideration above, we will not split it into training and test data for another model validation or test.

### - Summary Statistics

In [ ]:
summary_stats_Y <- clean_data |>
    summarize(
        mean = mean(art_count, na.rm = TRUE),
        max = max(art_count, na.rm = TRUE),
        sd = sd(art_count, na.rm = TRUE)
    ) |>
    mutate(variable = "art_count") |>
    select(variable, everything())

new_rows <- data.frame(
    variable = c("population_density", "median_income", "gini_income", "certificate_density"),
    mean = c(mean(subset_data$population_density, na.rm = TRUE), mean(subset_data$median_income, na.rm = TRUE), mean(subset_data$gini_income, na.rm = TRUE), mean(subset_data$certificate_density, na.rm = TRUE)),
    max = c(max(subset_data$population_density, na.rm = TRUE), max(subset_data$median_income, na.rm = TRUE), max(subset_data$gini_income, na.rm = TRUE), max(subset_data$certificate_density, na.rm = TRUE)),
    sd = c(sd(subset_data$population_density, na.rm = TRUE), sd(subset_data$median_income, na.rm = TRUE), sd(subset_data$gini_income, na.rm = TRUE), sd(subset_data$certificate_density, na.rm = TRUE))
)

summary_stats <- summary_stats_Y |>
    rbind(new_rows)

summary_stats |>
   mutate_if(is.numeric, round, 4)

Here is the summary statistic table of the key variables we will use in your analysis.

## Model

• Model: explain and justify your regression model. Make sure you describe it completely
and carefully. Write down an equation for the model. A good rule of thumb is that you
should have roughly 1 main specification and 3-4 variations of the main model, handling
specification choices.
- What variables did you use, and why? What about the errors?
- How did you specify the model (e.g. interactions)? Why?
- Which specifications did you use? Why are they different?
- What coeﬀicients or outputs from the model will answer your question? How?
- Is there anything else we need to know about your model?

Table of Results: clear a clear and complete table (or tables) of results, showing the
estimated model and your specification results

Firstly, we fit the full model with all of our input variables. The equation for this regression model will be: 

 Y = β0 + β1X1 + β2X2 + β3X3 + β4X4 + ϵ
> PublicArtDensity = β0 ​+ β1​Income_median + β2Income_gini_index + β3​number_of_people_with_certificates  + β4population_density + residual

The error(residual) is the difference between the actual value of Y the value predicted by our model. It reflects the unexplained portion of our model.

In [ ]:
full_model <- lm(art_count~.-Neighbourhood, data=clean_data)

summary(full_model)

In [ ]:
tidy(summary(full_model))%>% mutate_if(is.numeric, round, 5)

- positive association
- negative
- - pvalues  (significance level of 0.05)
- intercept: other better inferential features? but good normality in the error term (see in the next section)

In [ ]:
vif_values <- vif(full_model)
print(vif_values)

With the VIF values for these variables, there is no high multicollinearity among them.

#### Variation model 1 - reduced model

In definition, two variables `mean_median_income` and `mean_gini_income` might have some correlation, let us try the model without the mean_gini_income.

The equation for this regression model will be: 

 Y = β0 + β1X1 + β2X2 + β3X3 + ϵ
> PublicArtDensity = β0 ​+ β1​Income_median + β2​number_of_people_with_certificates  + β3population_density + residual

In [ ]:
no_gini_model <- lm(art_count~.-Neighbourhood-mean_gini_income, data=clean_data)

tidy(summary(no_gini_model))%>% mutate_if(is.numeric, round, 5)

#### Variation model 2 - interaction model

Finally, we try the interaction model with the interaction term of mean_population_density and mean_certificate_density, including the interaction between the population and count of certificates.

The equation for this regression model will be: 

 Y = β0 + β1X1 + β2X2 + β3X3 + β4X2*X3 + ϵ
> PublicArtDensity = β0 ​+ β1​Income_median + β2​number_of_people_with_certificates + β3population_density + β4mean_population_density*mean_certificate_density + residual

In [ ]:
interaction_model <- lm(art_count~mean_median_income+mean_population_density*mean_certificate_density, data=clean_data)

tidy(summary(interaction_model))%>% mutate_if(is.numeric, round, 5)

In [ ]:
robust_se <- sqrt(diag(vcovHC(interaction_model, type = "HC1")))
robust_se

## - Table of Results

In [ ]:
stargazer(full_model, no_gini_model, interaction_model, 
          type = "text", 
          covariate.labels = c("Mean Median Income", 
                               "Mean Population Density", 
                               "Mean Certificate Density", 
                               "Population Density * Certificate Density"), 
          dep.var.labels = c("Art Count"))

In [ ]:
adj_r2_full <- summary(full_model)$adj.r.squared
adj_r2_no_gini <- summary(no_gini_model)$adj.r.squared
adj_r2_interaction <- summary(interaction_model)$adj.r.squared

model_comparison <- data.frame(
  Model = c("Full Model", "No Gini Model", "Interaction Model"),
  Adjusted_R_squared = c(adj_r2_full, adj_r2_no_gini, adj_r2_interaction)
)

model_comparison

Our interaction model show the best adjusted R^2 score.

## Discussion

Discussion: discuss your results. Focus on how they answer your question, and what
they tell you - try to go beyond simply reading or reporting the table, and try to interpret
them. Provide additional tests to validate your model and the answers it provides, as
appropriate.
- You should provide at least (1) specification check for a key assumption necessary,
and at least one (1) alternative analysis or extensions (e.g. robustness) using a
regression model or related to help support or interpret your findings.

In the first model, we examine factors affecting the number of art installations (art_count). The results show that population density has a significant positive effect, with a coefficient of 0.006829 and a p-value of 1.03e-05. This suggests that denser areas tend to have more art installations, likely due to higher demand for public and cultural activities. However, median income and the Gini index are not statistically significant. Median income has a positive but weak effect (p-value = 0.0768), while the Gini index shows a large negative coefficient, indicating no significant impact. Certificate density also shows no significant effect (p-value = 0.1532). The model explains about 70% of the variation in art installations, with a statistically significant overall model (F-statistic = 13.14, p < 0.01).

In the "no_gini_model," we excluded the variable mean_gini_income, which might have correlated with mean_median_income. We attempt to improve the model by reducing potential multicollinearity. 
In this model, the estimation of mean_population_density is similar to model 1. We mainly focus on other variables. 
Mean_median_income has a coefficient of 0.00086 and a p-value is statistically significant at the 0.05 level. This is a stronger effect compared to the first model. By removing mean_gini_income, we have slightly improved the statistical significance of mean_median_income, suggesting that its positive relationship with art installations is more pronounced when multicollinearity between the two variables is avoided.Same as above, mean_certificate_density still doesn’t show a clear effect on art installations after excluding mean_gini_income.

In the interaction model, we explore how the interaction between population density and certificate density influences the number of public art installations. Here, all explantory variables show significant results. Notably, the coefficient for mean_certificate_density is now positive (678.167), which contrasts with its non-significant effect in the previous models. This could indicate that certificate density has a non-linear relationship with public art installations or there's no correlation. The interaction term between population density and certificate density is negative (-0.08446), suggesting that while both factors independently contribute positively to the number of public art installations, their combined effect is negative. In other words, as population density increases, but certificate density remains constant, the number of art installations decreases. Similarly, when certificate density increases but population density stays the same, the number of installations also decreases. This interaction term is difficult to interpret but highlights the complex relationship between these variables.

Overall, the interaction model provides the best explanation for the variation in art installations, as it includes significant coefficients and a high adjusted R² value of 0.885, showing a much better fit compared to the first model (R² = 0.69). 

## Conclusion


In conclusion, this study examined the factors affecting the placement of public art installations in Vancouver. Our findings show that mean median income and certificant density are important factors. Areas with higher income and population density tend to have more public art. The interaction between population density and certificate density was also significant in the third model, but in a negative direction. These results suggest that richer, denser neighborhoods are more likely to have public art, confirming previous research on how such art promotes community and engagement.

## References

1. Public art. (2024). Vancouver.ca. https://opendata.vancouver.ca/explore/dataset/public-art/export/?location=13,49.26603,-123.14601
(2021). Censusmapper.ca. https://censusmapper.ca/api/CA21#13/49.2717/-123.1262

2. Schuermans, N., Loopmans, M. P. J., & Vandenabeele, J. (2012). Public space, public art and public pedagogy. Social & Cultural Geography, 13(7), 675–682. https://doi.org/10.1080/14649365.2012.728007
